# Logistic Regression Coding Challenge

© Explore Data Science Academy

## Honour Code

I **YOUR NAME**, **YOUR SURNAME**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

## Overview

Within this coding challenge, we begin our practical experience of building models for classification problems. We do so with a basic Logistic Regression model.   

<br></br>

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/credit_card.jpg"
     alt="Learn good habits to avoid modeling debt"
     style="float: center; padding-bottom=0.5em"
     width=600px/>
Learn good habits to avoid modeling debt... Photo by <a href="https://unsplash.com/@rupixen?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText"> Rupixen.com </a> on Unsplash.
</div>

The structure of this notebook is as follows:

 - First, we will start off by loading and viewing the dataset.
 - We will see that the dataset has a mixture of both numerical and non-numerical features; that it contains values from different ranges; and that it contains a number of missing entries.
 - Based upon the observations above, we will preprocess the dataset to ensure the machine learning model we choose can make good predictions.
 - After our data is in good shape, we will do some exploratory data analysis to build our intuitions.
 - Finally, we will build a machine learning model that can predict if an individual's application for a credit card will be accepted.

### Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score


## The Dataset
We'll use the [Credit Card Approval dataset](http://archive.ics.uci.edu/ml/datasets/credit+approval) from the UCI Machine Learning Repository.
    
We explore the variables within this dataset in the sections below. 

### Reading in the data

First, loading and viewing the dataset. We find that since this data are confidential, the contributor of the dataset has anonymized the feature names.

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/89fee4463f428f55d31a254924e18501a3c468c3/Data/classification_sprint/cc_approvals.data',header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


The output may appear a bit confusing at its first sight, but let's try to figure out the most important features of a credit card application. The features of this dataset have been anonymized to protect the privacy, but [this blog](http://rstudio-pubs-static.s3.amazonaws.com/73039_9946de135c0a49daa7a0a9eda4a67a72.html) gives us a pretty good overview of the probable features. The probable features in a typical credit card application are <code>Gender</code>, <code>Age</code>, <code>Debt</code>, <code>Married</code>, <code>BankCustomer</code>, <code>EducationLevel</code>, <code>Ethnicity</code>, <code>YearsEmployed</code>, <code>PriorDefault</code>, <code>Employed</code>, <code>CreditScore</code>, <code>DriversLicense</code>, <code>Citizen</code>, <code>ZipCode</code>, <code>Income</code> and finally the <code>ApprovalStatus</code>. 

This gives us a pretty good starting point, and we can map these features with respect to the columns in the output.   

As we can see from our first glance at the data, the dataset has a mixture of numerical and non-numerical features. This can be fixed with some preprocessing.


<li>Our dataset contains both numeric and non-numeric data (specifically data that are of <code>float64</code>, <code>int64</code> and <code>object</code> types). Specifically, the features 2, 7, 10 and 14 contain numeric values (of types float64, float64, int64 and int64 respectively) and all the other features contain non-numeric values.</li>
<li>The dataset also contains values from several ranges. Some features have a value range of 0 - 28, some have a range of 2 - 67, and some have a range of 1017 - 100000.
<li>Finally, the dataset has missing values, which we'll take care of in this task. The missing values in the dataset are labeled with '?', which can be seen in the last cell's output.</li>
</ul>

## Data Cleaning

## Question 1 

Write a function to clean the given data . The function should:
* Replace the '?'s with NaN.
* Impute the missing values with mean imputation.
* Impute the missing values of non-numeric columns with the most frequent values as present in the respective columns.

_**Function Specifications:**_
* Should take a pandas Dataframe and column name as input and return a list as an output.
* The list should be a count of unique values in the column

In [88]:
### START FUNCTION
def data_cleaning(data, column_name):
    # replacing the ? with Nan
    data[column_name] = data[column_name].replace('?',np.nan)
    
    # getting an object and filling with mode
    if data[column_name].dtype == object:
        data[column_name]= data[column_name].fillna(value = data[column_name].mode()[0])
    # getting an object and filling with mean
    else:
        data[column_name] = data[column_name].fillna(value = data[column_name].mean())

    return list(df[column_name].value_counts())


### END FUNCTION

In [89]:
data_cleaning(df, 3)

[525, 163, 2]

_**Expected Outputs:**_
    

>```
data_cleaning(df, 0) == [480, 210]
data_cleaning(df, 9) == [395, 295]
```

## Data Preprocessing

## Question 2

Write a function to pre-process the data so that we can run it through the classifier. The function should:
* Convert the non-numeric data into numeric using sklearn's ```labelEncoder``` 
* Drop the features 11 and 13 and convert the DataFrame to a NumPy array
* Split the data into features and labels
* Standardise the features using sklearn's ```MinMaxScaler```
* Split the data into 80% training and 20% testing data.
* Use the `train_test_split` method from `sklearn` to do this.
* Set random_state to equal 42 for this internal method. 

_**Function Specifications:**_
* Should take a dataframe as input.
* Should return two `tuples` of the form `(X_train, y_train), (X_test, y_test)`.

In [67]:
### START FUNCTION
def data_preprocess(df):  
    
    # drop columns 11 and 13
    df = df.drop([11, 13], axis = 1)
    
    # select object type columns
    c = df.select_dtypes('object')
    
    # introduce label encoder chaning from categorical to numerical
    for col in c.columns:
        df[col] = LabelEncoder().fit_transform(df[col])
    
    #get the x and y of the data
    X = df.drop(15, axis = 1).to_numpy()
    y = df[15].to_numpy()
    
    
    #min max scaler
    minmax_scale = preprocessing.MinMaxScaler().fit(X)
    df_minmax = minmax_scale.transform(X)
    
    # split into train test sets
    X_train, X_test, y_train, y_test = train_test_split(df_minmax, y.astype(float) , test_size=0.2, random_state= 42)
    

    return (X_train, y_train), (X_test, y_test)

### END FUNCTION

In [68]:
(X_train, y_train), (X_test, y_test) = data_preprocess(df)


In [81]:
(X_train, y_train), (X_test, y_test) = data_preprocess(df)
print(X_train[:2])
print(y_train[:2])
print(X_test[:2])
print(y_test[:2])

[[1.         0.25787966 0.48214286 1.         1.         0.42857143
  0.33333333 0.         0.         0.         0.         0.
  0.        ]
 [1.         0.39255014 0.125      0.66666667 0.33333333 0.92857143
  0.88888889 0.12280702 1.         1.         0.04477612 0.
  0.        ]]
[1. 1.]
[[0.5        1.         0.05357143 0.66666667 0.33333333 0.42857143
  0.33333333 0.         0.         1.         0.02985075 0.
  0.00105   ]
 [0.5        0.7765043  0.14285714 0.66666667 0.33333333 0.57142857
  0.55555556 0.         1.         0.         0.         0.
  0.0096    ]]
[1. 0.]


_**Expected Outputs:**_

```python
(X_train, y_train), (X_test, y_test) = data_preprocess(df)
print(X_train[:2])
print(y_train[:2])
print(X_test[:2])
print(y_test[:2])
```

> ```
[[1.         0.25787966 0.48214286 1.         1.         0.42857143
  0.33333333 0.         0.         0.         0.         0.
  0.        ]]
[1.]
[[0.5        1.         0.05357143 0.66666667 0.33333333 0.42857143
  0.33333333 0.         0.         1.         0.02985075 0.
  0.00105   ]]
[1.]
```

## Training the model

## Question 3.1

Now that we have formatted our data, we can fit a model using sklearn's `LogisticRegression` class with solver 'lbfgs'. Write a function that will take as input `(X_train, y_train)` that we created previously, and return a trained model.

_**Function Specifications:**_
* Should take two numpy `arrays` as input in the form `(X_train, y_train)`.
* The returned model should be fitted to the data.

In [54]:
### START FUNCTION
def train_model(X_train, y_train):
    
    # get the logistic regression object
    lr = LogisticRegression()
    
    return lr.fit(X_train, y_train)

### END FUNCTION

In [55]:
lm = train_model(X_train, y_train)
print(lm.intercept_[0])
print(lm.coef_)

1.5068926456004823
[[ 0.25237869 -0.22847881 -0.01779302  2.00977742  0.23903441 -0.29504922
  -0.08952344 -0.83468871 -3.48756932 -1.07648711 -0.83688921  0.07860585
  -1.3077735 ]]


_**Expected Outputs:**_

```python
lm = train_model(X_train, y_train)
print(lm.intercept_[0])
print(lm.coef_)
```
```
1.5068926456005878
[[ 0.25237869 -0.22847881 -0.01779302  2.00977742  0.23903441 -0.29504922
  -0.08952344 -0.83468871 -3.48756932 -1.07648711 -0.83688921  0.07860585
  -1.3077735 ]]
```

## Testing the model

### Question 3.2 

AUC - ROC curve is a performance measurement for classification problem at various thresholds settings. ROC is a probability curve and AUC represents degree or measure of separability. It tells how much model is capable of distinguishing between classes. Write a function which returns the roc auc score of your trained model when tested with the test set.

_**Function Specifications:**_
* Should take the fitted model and two numpy `arrays` `X_test, y_test` as input.
* Should return a `float` of the roc auc score of the model. This number should be between zero and one.

_**Hint**_  Use the positive class's probability as the score

In [100]:
### START FUNCTION
def roc_score(lm, X_test, y_test):
    
    # predict the positive and negative propabilities
    probs = lm.predict_proba(X_test)
    
    # calculate AUC
    auc = roc_auc_score(y_test, probs[:,1])
    
    return auc

### END FUNCTION

In [101]:
print(roc_score(lm,X_test,y_test))

0.8865546218487395


_**Expected Outputs:**_
    
```python
print(roc_score(lm,X_test,y_test))
```
>```
0.8865546218487395
```

### Question 3.3

Write a function which calculates the Accuracy, Precision, Recall and F1 scores.

_**Function Specifications:**_
* Should take the fitted model and two numpy `arrays` `X_test, y_test` as input.
* Should return a tuple in the form (`Accuracy`, `Precision`, `Recall`, `F1-Score`)

In [104]:
### START FUNCTION
def scores(lm, X_test, y_test):
    # your code here
    
    accuracy = accuracy_score(y_test, lm.predict(X_test))
    
    precision = precision_score(y_test, lm.predict(X_test))
    
    recall = recall = recall_score(y_test, lm.predict(X_test))
    
    score = f1_score(y_test, lm.predict(X_test))
    
    return (accuracy, precision, recall, score)

### END FUNCTION

In [105]:
(accuracy, precision, recall, f1) = scores(lm, X_test, y_test)    

print('Accuracy: %f' % accuracy)
print('Precision: %f' % precision)
print('Recall: %f' % recall)
print('F1 score: %f' % f1)

Accuracy: 0.833333
Precision: 0.846154
Recall: 0.808824
F1 score: 0.827068


_**Expected Outputs:**_
```python
(accuracy, precision, recall, f1) = scores(lm,X_test,y_test)
    
print('Accuracy: %f' % accuracy)
print('Precision: %f' % precision)
print('Recall: %f' % recall)
print('F1 score: %f' % f1)
```
> ```
Accuracy: 0.833333
Precision: 0.846154
Recall: 0.808824
F1 score: 0.827068
```